<a href="https://colab.research.google.com/github/peterbabulik/ETA/blob/main/CHSHCircuit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Correlation & Causality: Bell's Theorem

**The Math:** The CHSH Inequality.
$$ S = E(a,b) - E(a,b') + E(a',b) + E(a',b') \leq 2 $$

Classically (local hidden variables), this sum cannot exceed 2. Quantumly, it can reach $2\sqrt{2} \approx 2.82$.

### The Quantum Translation: The CHSH Circuit

This is less of a "computation" and more of a **"unit test for reality."** It proves that the qubits are entangled and not just sharing pre-agreed random numbers.

**The Logic:**
1. Create Entanglement (Hadamard + CNOT) - a Bell state $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$.
2. Alice measures in basis $A$ or $A'$.
3. Bob measures in basis $B$ or $B'$.
4. Calculate correlation $E$ for each basis combination.
5. Compute $S$ and verify $S > 2$ (violation of classical bounds).

**Analogy for Python Devs:**
Think of this as `test_reality()` - a unit test that proves quantum entanglement is real and not just classical correlation.

### The Qiskit Implementation

We'll create a Bell state and measure in different bases to compute the CHSH value.

In [1]:
!pip install qiskit qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.8 MB/s eta 0:00:00


In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def create_bell_state():
    """
    Creates a Bell state (maximally entangled state):
    |Φ+⟩ = (|00⟩ + |11⟩) / √2

    Circuit:
    1. H on qubit 0: |00⟩ -> (|00⟩ + |10⟩) / √2
    2. CNOT(0, 1): -> (|00⟩ + |11⟩) / √2
    """
    qc = QuantumCircuit(2)
    qc.h(0)      # Hadamard on Alice's qubit
    qc.cx(0, 1)  # CNOT: Alice controls Bob's qubit
    return qc

def measure_in_basis(qc, qubit, angle):
    """
    Measures a qubit in a rotated basis.

    Standard measurement is in Z basis (computational basis).
    To measure in a different basis, we rotate the qubit first.

    For angle θ, we measure in the basis:
    |+θ⟩ = cos(θ/2)|0⟩ + sin(θ/2)|1⟩
    |-θ⟩ = -sin(θ/2)|0⟩ + cos(θ/2)|1⟩

    This is achieved by applying Ry(-θ) before measurement.
    """
    qc.ry(-angle, qubit)

def run_chsh_experiment(shots=10000):
    """
    Runs the complete CHSH experiment.

    We measure in 4 different basis combinations:
    - (A, B):   Alice at 0°,   Bob at 45°
    - (A, B'):  Alice at 0°,   Bob at 135°
    - (A', B):  Alice at 90°,  Bob at 45°
    - (A', B'): Alice at 90°,  Bob at 135°

    These angles are chosen to maximize the CHSH violation.
    """
    # Define measurement angles (in radians)
    # Alice's bases
    a = 0           # 0°
    a_prime = np.pi/2  # 90°

    # Bob's bases
    b = np.pi/4     # 45°
    b_prime = 3*np.pi/4  # 135°

    # Store results for each basis combination
    results = {}

    simulator = AerSimulator()

    # Run experiments for each basis combination
    basis_combinations = [
        ('AB', a, b),
        ('AB\'', a, b_prime),
        ('A\'B', a_prime, b),
        ('A\'B\'', a_prime, b_prime)
    ]

    for name, alice_angle, bob_angle in basis_combinations:
        # Create fresh Bell state
        qc = create_bell_state()

        # Apply measurement rotations
        measure_in_basis(qc, 0, alice_angle)  # Alice's qubit
        measure_in_basis(qc, 1, bob_angle)    # Bob's qubit

        # Measure both qubits
        qc.measure_all()

        # Run simulation
        compiled = transpile(qc, simulator)
        job = simulator.run(compiled, shots=shots)
        counts = job.result().get_counts()

        results[name] = counts

    return results

def calculate_correlation(counts, shots):
    """
    Calculates the correlation coefficient E.

    E = P(00) + P(11) - P(01) - P(10)

    This measures how correlated the two measurements are:
    - E = +1: Perfectly correlated (always same outcome)
    - E = -1: Perfectly anti-correlated (always opposite)
    - E = 0: No correlation
    """
    p00 = counts.get('00', 0) / shots
    p01 = counts.get('01', 0) / shots
    p10 = counts.get('10', 0) / shots
    p11 = counts.get('11', 0) / shots

    return p00 + p11 - p01 - p10

def calculate_chsh_value(results, shots):
    """
    Calculates the CHSH S value.

    S = E(a,b) - E(a,b') + E(a',b) + E(a',b')

    Classical limit: |S| ≤ 2
    Quantum limit: |S| ≤ 2√2 ≈ 2.82
    """
    correlations = {}
    for name, counts in results.items():
        correlations[name] = calculate_correlation(counts, shots)

    # S = E(a,b) - E(a,b') + E(a',b) + E(a',b')
    S = (correlations['AB'] - correlations['AB\''] +
         correlations['A\'B'] + correlations['A\'B\''])

    return S, correlations

# --- Run the CHSH Experiment ---

print("=== CHSH Inequality Experiment ===")
print("Testing Bell's Theorem with Quantum Entanglement\n")

shots = 10000
results = run_chsh_experiment(shots)

# Calculate CHSH value
S, correlations = calculate_chsh_value(results, shots)

print("--- Measurement Results ---")
for name, counts in results.items():
    print(f"{name}: {counts}")

print("\n--- Correlation Coefficients ---")
for name, E in correlations.items():
    print(f"E({name}) = {E:.4f}")

print("\n--- CHSH Value ---")
print(f"S = E(AB) - E(AB') + E(A'B) + E(A'B')")
print(f"S = {correlations['AB']:.4f} - {correlations['AB\'']:.4f} + {correlations['A\'B']:.4f} + {correlations['A\'B\'']:.4f}")
print(f"S = {S:.4f}")

print("\n--- Interpretation ---")
print(f"Classical limit: |S| ≤ 2")
print(f"Quantum limit:   |S| ≤ 2√2 ≈ {2*np.sqrt(2):.4f}")
print(f"Measured value:  S = {S:.4f}")

if abs(S) > 2:
    print(f"\n✓ CHSH INEQUALITY VIOLATED!")
    print(f"  |S| = {abs(S):.4f} > 2")
    print(f"  This proves quantum entanglement cannot be explained by local hidden variables.")
else:
    print(f"\n✗ No violation detected (this would be unexpected for a quantum system)")

=== CHSH Inequality Experiment ===
Testing Bell's Theorem with Quantum Entanglement

--- Measurement Results ---
AB: {'01': 709, '11': 4206, '10': 729, '00': 4356}
AB': {'11': 742, '10': 4335, '00': 675, '01': 4248}
A'B: {'01': 734, '11': 4260, '10': 770, '00': 4236}
A'B': {'01': 752, '11': 4224, '00': 4327, '10': 697}

--- Correlation Coefficients ---
E(AB) = 0.7124
E(AB') = -0.7166
E(A'B) = 0.6992
E(A'B') = 0.7102

--- CHSH Value ---
S = E(AB) - E(AB') + E(A'B) + E(A'B')
S = 0.7124 - -0.7166 + 0.6992 + 0.7102
S = 2.8384

--- Interpretation ---
Classical limit: |S| ≤ 2
Quantum limit:   |S| ≤ 2√2 ≈ 2.8284
Measured value:  S = 2.8384

✓ CHSH INEQUALITY VIOLATED!
  |S| = 2.8384 > 2
  This proves quantum entanglement cannot be explained by local hidden variables.


In [3]:
# Visualize the Bell state preparation circuit
print("--- Bell State Preparation Circuit ---")
bell_circuit = create_bell_state()
print(bell_circuit.draw(output='text'))

print("\n--- Circuit for CHSH Measurement (example: AB basis) ---")
qc = create_bell_state()
measure_in_basis(qc, 0, 0)        # Alice at 0°
measure_in_basis(qc, 1, np.pi/4)  # Bob at 45°
qc.measure_all()
print(qc.draw(output='text'))

--- Bell State Preparation Circuit ---
     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

--- Circuit for CHSH Measurement (example: AB basis) ---
        ┌───┐      ┌───────┐   ░ ┌─┐   
   q_0: ┤ H ├──■───┤ Ry(0) ├───░─┤M├───
        └───┘┌─┴─┐┌┴───────┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤ Ry(-π/4) ├─░──╫─┤M├
             └───┘└──────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════╩══╩═
                                  0  1 


### Understanding the Translation

1. **$S \leq 2$ (The Classical Bound):**
   - If particles have pre-determined values (local hidden variables), the CHSH sum cannot exceed 2.
   - This is a mathematical consequence of classical probability theory.

2. **$S = 2\sqrt{2}$ (The Quantum Value):**
   - Quantum mechanics predicts $S = 2\sqrt{2} \approx 2.82$ for maximally entangled states.
   - This violation has been experimentally confirmed numerous times.

3. **Entanglement:**
   - The Bell state $|\Phi^+\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}}$ creates quantum correlations.
   - Measuring one qubit instantly affects the state of the other.

### Why is this important?

**Fundamental Physics:** This experiment proves that nature cannot be described by local hidden variable theories. Einstein's "spooky action at a distance" is real.

**Quantum Technology:** Entanglement is the resource that powers:
- Quantum teleportation
- Quantum cryptography (BB84, E91 protocols)
- Quantum computing speedups
